In [1]:
import numpy as np
import os
import sys       
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy
import pickle

# Get the path of the parent_directory
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
parse_filtered = parent_dir + "/data/interim/parse_filtered"
outputdir = parent_dir +  "/models"
figures_dir = parent_dir + "/reports/figures"
sys.path.append(parent_dir)

# For .bin patch parsing
from patch.Processing.processECG import process_ECG
from patch.Processing.processPPG import process_PPG, select_ppg_array, extract_PPG_fiducial
from patch.Processing.processSCG import process_SCG, extract_SCG_fiducials
from patch.Tools.preprocessing import get_filt_dict, dict_interpolation
from patch.Tools.pipelineConfig import *
from patch.Tools.patchParser import get_newest_patch_file, parse_file

%load_ext autoreload
%autoreload 2
%matplotlib widget

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
sub_act_paths = os.listdir(parse_filtered)
sub_act_paths = [parse_filtered + '/' + s for s in sub_act_paths]

In [4]:
%matplotlib inline
act_titles = ["prep","recovery","rest_2","script","speaking"]

# Turn interactive plotting off
show_plots = False
plt.ioff()

for subject in sub_act_paths:
    prep = pickle.load(open(subject + "/prep.pkl", "rb"))
    recovery = pickle.load(open(subject + "/recovery.pkl", "rb"))
    rest_2 = pickle.load(open(subject + "/rest_2.pkl", "rb"))
    script = pickle.load(open(subject + "/script.pkl", "rb"))
    speaking = pickle.load(open(subject + "/speaking.pkl", "rb"))
    activities = [prep,recovery,rest_2,script,speaking]
    count = 0
    
    # For skipping specific subjects
    skip = False
    for sub in skip_sub:
        if sub in subject:
            skip = True
    if skip:
        continue
    
    print(f"Subject: {os.path.basename(os.path.normpath(subject))}")
    
    for activity in activities:
        # Output features directory
        out = outputdir+f'/{os.path.basename(os.path.normpath(subject))}/{act_titles[count]}/'
        
        # Output figures directory
        fig_dir = figures_dir+f'/{os.path.basename(os.path.normpath(subject))}/{act_titles[count]}/'
        
        print(f"Activity: {act_titles[count]}")
        
        count = count + 1
        
        # Creating directories that don't exist
        if not os.path.exists(out):
            os.makedirs(out)
        if not os.path.exists(fig_dir):
            os.makedirs(fig_dir)
        
        # Plotting raw signals
        fig, axs = plt.subplots(3,1, sharex=True, figsize=(12,8))
        plt.suptitle("Raw Signals")
        axs[0].plot(activity['time'], activity['ecg'], color='salmon')
        axs[1].plot(activity['time'], activity['scg_DV'], color='teal')
        axs[2].plot(activity['time'], activity['ppg_ir_2'], color='darkgreen')
        plt.savefig(fig_dir + "raw_signal.png")
        
        if show_plots:
            plt.show()

        # Note we pass the raw ECG here, as we use different R-peak detectors that each require specific filtering
        ecg_dict = process_ECG(activity['ecg'],
                        activity['time'],
                        force_inverse=False,
                        width_QRS=width_QRS,
                        peak_tol=pk_tolerance_ms, 
                        beat_len=beat_length, 
                        Fs=FS_RESAMPLE, 
                        verbose=True, 
                        plotFlag=True)
        
        # PPG Array selection
        ppg_arr,_ = select_ppg_array(activity[ppg_wavelen+'_1'], 
                                    activity[ppg_wavelen+'_2'], 
                                    verbose=True)

        # Process PPG
        ppg_dict = process_PPG(activity[ppg_wavelen+ppg_arr], 
                                activity[ppg_wavelen+ppg_arr], 
                                ecg_dict, 
                                ensembleSize=ensemble_length, 
                                useTemplateSQI=True,
                                beat_len=beat_length, 
                                Fs=FS_RESAMPLE, 
                                verbose=True)

        plt.figure(figsize=(9, 6))
        _=plt.plot(ppg_dict['ppg_beats'], color='gray', alpha=0.1)
        _=plt.plot(np.mean(ppg_dict['ppg_beats'], axis=1), color='red', alpha=0.6, lw=3)
        _=plt.savefig(fig_dir + "ppg_beats.png")
        
        # Process SCG
        scg_dict = process_SCG(activity['scg_DV'], 
                                activity['pcg_DV'], 
                                ecg_dict, 
                                ensembleSize=ensemble_length, 
                                useTemplateSQI=True,
                                beat_len=beat_length, 
                                Fs=FS_RESAMPLE, 
                                verbose=True)

        plt.figure(figsize=(12, 6))
        _=plt.plot(scg_dict['scg_beats'], color='gray', alpha=0.1)
        _=plt.plot(np.mean(scg_dict['scg_beats'], axis=1), color='red', alpha=0.6, lw=3)
        _=plt.savefig(fig_dir + "scg_beats.png")
        
        scg_fid_dict = extract_SCG_fiducials(scg_dict, 
                                    num_features_ao=4, 
                                    num_features_ac=3, 
                                    ema_factor=1.0, 
                                    verbose=True, 
                                    method = 'Template') # choose between 'GMM', 'Template', or 'Jon'

        ao_locs, ac_locs, flagged_ao, flagged_ac = np.squeeze(scg_fid_dict['ao_locs']), np.squeeze(scg_fid_dict['ac_locs']), scg_fid_dict['flagged_ao'], scg_fid_dict['flagged_ac']
        scg_beats_used = scg_fid_dict['beats_used']
        # Update these as they may have been reduced to find possible candidates
        num_feats_ao, num_feats_ac = np.shape(scg_fid_dict['ao_cands'])[0], np.shape(scg_fid_dict['ac_cands'])[0]

        # Plot results of fiducial tracking
        scg_beats = scg_dict['scg_beats'].copy()

        print(f"Least deviation ao_loc={np.argmin(np.nanstd(scg_fid_dict['ao_cands'], axis=1))}")
        if num_feats_ac > 1:
            max_idx = np.nanmax(scg_fid_dict['ac_cands'][-1].T)
            print(f"Least deviation ac_loc={np.argmin(np.nanstd(scg_fid_dict['ac_cands'], axis=1))}")
        else:
            max_idx = np.nanmax(scg_fid_dict['ac_cands'])
            print(f"Least deviation ac_loc={np.argmin(np.nanstd(scg_fid_dict['ac_cands']))}")
        scg_beats = np.ceil(255 * (scg_beats / scg_beats.max(axis=0)))

        f, ax = plt.subplots(1, 2, figsize=(14, 5))
        ax[0].imshow(scg_beats[:int(max_idx),:], cmap='gray',vmin = 0, vmax = 255, aspect='auto')
        ax[0].plot(scg_fid_dict['ao_cands'].T, color='red', alpha=0.3)
        ax[0].plot(scg_fid_dict['ac_cands'].T, color='red', alpha=0.3)
        ax[0].plot(ao_locs.T, color='blue', alpha=0.8, label = 'AO')
        ax[0].plot(ac_locs.T, color='g', alpha=0.8, label = 'AC')
        ax[0].plot(scg_fid_dict['s1_loc']*np.ones(len(scg_fid_dict['ao_locs'])), color='green', lw=3, alpha=0.2)
        ax[0].set_ylabel('Time from R-peak [samples]')
        ax[0].set_xlabel('Beats')
        ax[0].legend()

        ao_amp = [scg_dict['scg_beats'][ao_loc_i, scg_beat_idx][0] for ao_loc_i, scg_beat_idx in zip(scg_fid_dict['ao_locs'].astype(int), scg_fid_dict['flagged_ao'])]
        ac_amp = [scg_dict['scg_beats'][ac_loc_i, scg_beat_idx][0] for ac_loc_i, scg_beat_idx in zip(scg_fid_dict['ac_locs'].astype(int), scg_fid_dict['flagged_ac'])]
        ax[1].plot(scg_dict['scg_beats'], color='gray', alpha=0.1)
        ax[1].plot(np.mean(scg_dict['scg_beats'], axis=1), color='red', alpha=0.6, lw=3)
        ax[1].scatter(scg_fid_dict['ao_locs'], ao_amp, alpha = 0.3, color = 'tab:blue', label = 'AO')
        ax[1].scatter(scg_fid_dict['ac_locs'], ac_amp, alpha = 0.3, color = 'tab:green', label = 'AC')
        ax[1].set_xlabel('Time from R-peak [samples]')
        ax[1].set_ylabel('SCG amplitude')
        ax[1].legend()
        plt.savefig(fig_dir + "scg_fiducial.png")
        if show_plots:
            plt.show()
        
        ppg_fid_dict = extract_PPG_fiducial(ppg_dict, memory=30, smooth=10, max_dist_foot=max_dist_foot)

        ppg_beats = ppg_dict['ppg_beats'].copy()
        ppg_beats = np.ceil(255 * (ppg_beats / ppg_beats.max(axis=0)))

        f, ax = plt.subplots(1, 2, figsize=(14, 5))
        ax[0].imshow(ppg_beats, cmap='gray',vmin = 0, vmax = 255, aspect='auto')
        ax[0].plot(ppg_fid_dict['pulse_locs'], color='blue', alpha=0.6, label = 'PAT')
        ax[0].set_ylabel('Time from R-peak [samples]')
        ax[0].set_xlabel('Beats')
        ax[0].legend()


        ppg_pat_amp = [ppg_dict['ppg_beats'][pat_loc_i, i] for i, pat_loc_i in  enumerate(ppg_fid_dict['pulse_locs'].astype(int))]
        ax[1].plot(ppg_dict['ppg_beats'], color='gray', alpha=0.1)
        ax[1].plot(np.mean(ppg_dict['ppg_beats'], axis=1), color='red', alpha=0.6, lw=3)
        ax[1].scatter(ppg_fid_dict['pulse_locs'], ppg_pat_amp, alpha = 0.3, color = 'tab:blue', label = 'PAT')
        ax[1].set_xlabel('Time from R-peak [samples]')
        ax[1].set_ylabel('PPG amplitude')
        ax[1].legend()
        plt.savefig(fig_dir + "ppg_fiducial.png")
        if show_plots:
            plt.show()
        
        feat_df = pd.DataFrame()

        t_feats = ecg_dict['t_beats']
        feat_df['time'] = t_feats

        feat_df['hr'] = ecg_dict['hr']

        # Prerequisite check to see if greater than 5 minutes of data is present (necessary to compute all HRV features)
        if (feat_df['time'].values[-1] - feat_df['time'].values[0]) < 5*60:
            print('Not enough data for HRV analysis')
            # Frequency-domain
            feat_df['hrv_lf'], feat_df['hrv_hf'], feat_df['hrv_lf'] = np.nan, np.nan, np.nan
            # Time-domain
            feat_df['hrv_rmssd'], feat_df['hrv_sd'], feat_df['hrv_sdsd'], feat_df['hrv_sdnn'] = np.nan, np.nan, np.nan, np.nan
        # If there is enough data, compute HRV features
        # else:
        #   # Frequency-domain

        #   # Time-domain

        feat_PEP = np.squeeze((scg_fid_dict['ao_locs'][scg_fid_dict['flagged_ao']])*1000/FS_RESAMPLE)
        ao_times = np.intersect1d(scg_fid_dict['t_ao'], feat_df.time.values)
        # Add PEP values to df
        for i, ao_time in enumerate(ao_times):
            feat_df.loc[feat_df.time == ao_time, 'pep'] = feat_PEP[i]

        # Get beats with defined AO and AC points
        ac_times = np.intersect1d(scg_fid_dict['t_ac'], feat_df.time.values)
        _, ao_ind, ac_ind = np.intersect1d(ao_times, ac_times, return_indices=True)
        pep_lvet_times = ao_times[ao_ind]
        feat_PEP_LVET = np.squeeze((ao_locs[ao_ind])*1000/FS_RESAMPLE) / np.squeeze((ac_locs[ac_ind])*1000/FS_RESAMPLE)
        # Add PEP/LVET values to df
        for i, pep_lvet_time in enumerate(pep_lvet_times):
            feat_df.loc[feat_df.time == pep_lvet_time, 'pep_lvet'] = feat_PEP_LVET[i]
            
        feat_LVET = np.squeeze((ac_locs[ac_ind])*1000/FS_RESAMPLE) - np.squeeze((ao_locs[ao_ind])*1000/FS_RESAMPLE)
        # Add LVET values to df
        for i, ac_time in enumerate(pep_lvet_times):
            feat_df.loc[feat_df.time == ac_time, 'lvet'] = feat_LVET[i]
            
        ppg_amp_ac = ppg_dict['ppg_amp_ac']
        ppg_amp_ac_dc = ppg_dict['ppg_amp'] # AC amplitude normalized by DC (in case of changes in contact pressure)
        t_ppg_amp = ppg_dict['t_ppg_amp']
        # Add PPG amplitude values to df
        for i, ppg_amp_time in enumerate(t_ppg_amp):
            feat_df.loc[feat_df.time == ppg_amp_time, 'ppg_amp_ac'] = ppg_amp_ac[i]
            feat_df.loc[feat_df.time == ppg_amp_time, 'ppg_amp'] = ppg_amp_ac_dc[i]
            
        feat_PAT = ppg_fid_dict['pulse_locs']
        t_pat = ppg_fid_dict['t_pulse']
        # Add PAT values to df
        for i, pat_time in enumerate(t_pat):
            feat_df.loc[feat_df.time == pat_time, 'pat'] = feat_PAT[i]

        # Find overlapping beats with PAT and PEP values for PTT
        _, pat_ind, pep_ind = np.intersect1d(t_pat, ao_times, return_indices=True)
        pat_for_ptt = feat_PAT[pat_ind]
        pep_for_ptt = feat_PEP[pep_ind]
        feat_PTT = pat_for_ptt - pep_for_ptt
        t_ptt = t_pat[pat_ind]
        # Add PTT values to df
        for i, ptt_time in enumerate(t_ptt):
            feat_df.loc[feat_df.time == ptt_time, 'ptt'] = feat_PTT[i]
            
        # print(feat_df)

        f, ax = plt.subplots(3, 1, figsize = (16, 6), sharex = True)

        # Plot r-peak (ecg) features
        r_peak_idx = [np.argmin(np.abs(activity['time'] - rpeak_i)) for rpeak_i in feat_df['time']]
        ax[0].plot(activity['time'], activity['ecg'], color='salmon')
        ax[0].scatter(feat_df['time'], activity['ecg'][r_peak_idx], color = 'tab:blue', label = 'R-peak')
        ax[0].legend()
        ax[0].set_ylabel('ecg')

        # Plot ao (scg) features
        ao_loc = feat_df['time'] + feat_df['pep']/1000
        ao_idx = [np.argmin(np.abs(activity['time'] - ao_loc_i)) for ao_loc_i in ao_loc]
        ax[1].plot(activity['time'], activity['scg_DV'], color='teal')
        ax[1].scatter(feat_df['time'] + feat_df['pep']/1000, activity['scg_DV'][ao_idx], color = 'tab:orange', label = 'AO')


        # Plot ac (scg) features
        ac_loc = feat_df['time'] + (feat_df['lvet'] +  feat_df['pep'])/1000
        ac_idx = [np.argmin(np.abs(activity['time'] - ac_loc_i)) for ac_loc_i in (ac_loc)]
        ax[1].scatter(ac_loc, activity['scg_DV'][ac_idx], color = 'tab:purple', label = 'AC')
        ax[1].set_ylim([-0.1, 0.1])
        ax[1].legend()
        ax[1].set_ylabel('scg')

        # Plot pat (ppg) features
        if 'pat' in feat_df:
            pat_loc = feat_df['time'] + feat_df['pat']/1000
            pat_idx = [np.argmin(np.abs(activity['time'] - pat_loc_i)) for pat_loc_i in (pat_loc)]
            ax[2].plot(activity['time'], activity[ppg_wavelen+ppg_arr], color='darkgreen')
            ax[2].scatter(pat_loc, activity[ppg_wavelen+ppg_arr][pat_idx], color = 'k', label = 'ppg foot')
            ax[2].set_ylim([-200, 200])
            ax[2].legend()
            ax[2].set_ylabel('ppg')
            ax[2].set_xlabel('Time (s)')
        
        plt.savefig(fig_dir + "r_ao_ac_ppgfoot.png")
        if show_plots:
            plt.show()
        
        # if 'pat' in feat_df:    
        #     feats_to_plot = ['hr', 'pep', 'lvet', 'pep_lvet', 'ppg_amp_ac', 'pat', 'ptt']
        # else:
        #     feats_to_plot = ['hr', 'pep', 'lvet', 'pep_lvet', 'ppg_amp_ac', 'ptt']

        # feat_units = ['bpm', 'ms', 'ms', 'a.u.', 'uW', 'ms', 'ms']
        # colors = sns.color_palette(palette='Set2', n_colors=len(feats_to_plot))
        # fig, axs = plt.subplots(len(feats_to_plot), 1, sharex=True, figsize=(16,14))
        # for ax, feat, unit, col in zip(axs, feats_to_plot, feat_units, colors):
        #     ax.scatter(feat_df.time, feat_df.loc[:,feat], label=feat, color=col)
        #     ax.set_ylabel(unit)
        #     ax.legend(loc='upper right')
        # plt.xlabel('Time (s)')
        
        # plt.savefig(fig_dir + "features.png")
        # plt.show()
        
        feat_df.to_csv(out+f'output_features.csv', index = False)

Subject: 3161
Activity: prep
ECG signal was inverted.
Number of peaks found:
   Neurokit (191)
   Martinez (192)
   Kalidas (191)
   Common (191)
187 NN intervals detected from 191 peaks.
Using PPG array 2
Begin processing PPG
Stage 1: Global MAD removed 21 out of 187
Stage 2: Moving window MAD removed 5 out of 166
Stage 3: Moving window DTW removed 41 out of 161
Stage 4: Global DTW fusion removed 37 out of 161
Final: Using 153 beats out of 187
Begin processing SCG...
Stage 1: Global MAD removed 5 out of 187
Stage 2: Moving window MAD removed 1 out of 182
Stage 3: Moving window DTFM removed 43 out of 181
Stage 4: Global DTFM removed 0 out of 181
Final: Using 181 beats out of 187
Engine started...
Least deviation ao_loc=3
Least deviation ac_loc=0
Not enough data for HRV analysis
Activity: recovery
ECG signal was inverted.
Number of peaks found:
   Neurokit (1880)
   Martinez (1880)
   Kalidas (1880)
   Common (1880)
1838 NN intervals detected from 1880 peaks.
Using PPG array 1
Begin pro

/tmp/ipykernel_3981072/1820057131.py:97: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 6))


Engine started...
Least deviation ao_loc=3
Least deviation ac_loc=0
Activity: script
ECG signal was inverted.
Number of peaks found:
   Neurokit (194)
   Martinez (194)
   Kalidas (194)
   Common (193)
190 NN intervals detected from 193 peaks.
Using PPG array 1
Begin processing PPG
Stage 1: Global MAD removed 16 out of 190
Stage 2: Moving window MAD removed 1 out of 174
Stage 3: Moving window DTW removed 163 out of 173
Stage 4: Global DTW fusion removed 84 out of 173
Final: Using 173 beats out of 190
Begin processing SCG...
Stage 1: Global MAD removed 8 out of 190
Stage 2: Moving window MAD removed 1 out of 182
Stage 3: Moving window DTFM removed 57 out of 181
Stage 4: Global DTFM removed 2 out of 181
Final: Using 179 beats out of 190
Engine started...
Least deviation ao_loc=1
Least deviation ac_loc=0
Not enough data for HRV analysis
Activity: speaking
ECG signal was inverted.
Number of peaks found:
   Neurokit (284)
   Martinez (284)
   Kalidas (284)
   Common (284)
277 NN intervals d

/home/michael/miniconda3/envs/mims-transformer-stress-classification/lib/python3.11/site-packages/neurokit2/signal/signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2


ECG signal was inverted.
Number of peaks found:
   Neurokit (199)
   Martinez (2)
   Kalidas (198)
   Common (122)
80 NN intervals detected from 122 peaks.
Using PPG array 2
Begin processing PPG
Stage 1: Global MAD removed 9 out of 80
Stage 2: Moving window MAD removed 0 out of 71
Stage 3: Moving window DTW removed 0 out of 71
Stage 4: Global DTW fusion removed 0 out of 71
Final: Using 71 beats out of 80
Begin processing SCG...
Stage 1: Global MAD removed 0 out of 80
Stage 2: Moving window MAD removed 0 out of 80
Stage 3: Moving window DTFM removed 0 out of 80
Stage 4: Global DTFM removed 0 out of 80
Final: Using 80 beats out of 80
Engine started...
Least deviation ao_loc=0
Least deviation ac_loc=0
Not enough data for HRV analysis
Activity: recovery
Number of peaks found:
   Neurokit (1872)
   Martinez (1872)
   Kalidas (1872)
   Common (1854)
1619 NN intervals detected from 1854 peaks.
Using PPG array 1
Begin processing PPG
Stage 1: Global MAD removed 179 out of 1619
Stage 2: Moving w

/home/michael/miniconda3/envs/mims-transformer-stress-classification/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/michael/miniconda3/envs/mims-transformer-stress-classification/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Number of peaks found:
   Neurokit (1220)
   Martinez (0)
   Kalidas (1219)
   Common (0)


/home/michael/miniconda3/envs/mims-transformer-stress-classification/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


IndexError: index -1 is out of bounds for axis 0 with size 0